<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:30px; text-align: center;"><font size=4 color=blue><b>[프로젝트] 학습목표</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
○ 객체 탐지 방법에 대해 학습한다.<br><br>
○ yolo 기반 라이브러리를 활용한 객체 탐지 방법에 대해 학습한다.<br><br>
○ yolov7을 활용하여 커스텀 데이터의 객체 탐지 방법에 대해 학습한다.
</font></td></tr>   
</table>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 객체 탐지

## 개요

- 참고 : https://pseudo-lab.github.io/Tutorial-Book/chapters/object-detection/Ch1-Object-Detection.html
- https://leedakyeong.tistory.com/entry/Object-Detection%EC%9D%B4%EB%9E%80-Object-Detection-%EC%9A%A9%EC%96%B4%EC%A0%95%EB%A6%AC


- 객체 탐지(object detection) : 컴퓨터 비전과 이미지 처리와 관련된 컴퓨터 기술로서, 디지털 이미지와 비디오에서 관심 객체(예: 인간, 건물, 자동차 등)의 위치를 감지하는 작업
  - 활용 분야 : 얼굴 검출, 보행자 검출, 영상 복구, 비디오 감시, 자율주행 등
  
<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_object_detect.PNG" width=30%> </center>
  
- 객체 분류 : 탐지된 객체가 무엇인지 판단하고 분류하는 작업

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_object_detect2.png" width=30%> </center>

- 객체 탐지 및 분류 알고리즘 (참고 : 위키백과)
  - 머신러닝 기반 접근 : 아래 알고리즘을 사용하여 정의하고 SVM 등을 이용하여 분류
    - Haar features 기반 비올라–존스 객체탐지 프레임워크
    - Scale-invariant feature transform (SIFT)
    - Histogram of oriented gradients (HOG) 기능
  - 딥러닝 기반 접근 : 구체적 정의가 필요 없고, 일반적으로 합성곱 신경망(CNN)을 활용
    - Region Proposals (R-CNN, Fast R-CNN, Faster R-CNN, cascade R-CNN)
    - Single Shot MultiBox Detector (SSD)
    - You Only Look Once (YOLO)
    - Single-Shot Refinement Neural Network for Object Detection (RefineDet)
    - Retina-Net
    - Deformable convolutional networks
    
  
<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_object_detect3.png" width=60%> </center>
<center><font size=1>참고 : Zou et al. 2019. Object Detection in 20 Years: A Survey</font></center>    


- Two-Stage Detector
  - Classification, Regional Proposal을 순차적으로 수행하여 결과를 얻는 방법
  - 객체를 검출하는 정확도 측면에서는 좋은 성능을 냈지만, 예측 속도가 느려 실시간 탐지에 제한
  - R-CNN, Fast R-CNN, Faster R-CNN 등

- One-Stage Detector :
  - Classification, Regional Proposal을 동시에 수행하여 결과를 얻는 방법    
  - Two-Stage Detector의 속도 문제를 해결하기 위해 제안
  - YOLO, SSD, RetinaNet 등
  
<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_object_detect4.png" width=60%> </center>
<center><font size=1>참고 : Naver BoostCamp AI Tech - edwith 강의</font></center>  

## 바운딩 박스 (Bounding Box)

- 객체 탐지 모델에서 바운딩 박스는 타겟 위치를 특정하기 위해 사용
  - 타겟 위치를 X와 Y축을 이용하여 사각형으로 표현
  - 예를 들어, 바운딩 박스 값은 (X 최소값, Y 최소값, X 최대값, Y 최대값)으로 표현
  
<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_bounding_box.PNG" width=40%> </center>
<center><font size=1>참고 : https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection</font></center>
  

- 일반적으로 효율성을 높이기 위해 바운딩 박스의 위치를 0-1값으로 정규화

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_bounding_box1.PNG" width=40%> </center>
<center><font size=1>참고 : https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection/raw/master/img/bc2.PNG</font></center>

- 객체 탐지 모델에 사용되는 데이터의 크기가 크기 때문에 바운딩 박스를 사용하여 객체의 위치를 설정하여 탐지하고 딥러닝 과정에서 바운딩 박스 영역만 활용

## Yolo (You only Look Once : 한 번보고 처리한다)

- 참고
  - https://blog.nerdfactory.ai/2021/05/06/You-Only-Look-Once.-YOLO.html
  - https://docs.google.com/presentation/d/1aeRvtKG21KHdD5lg6Hgyhx5rPq_ZOsGjG5rJ1HP7BbA/pub?start=false&loop=false&delayms=3000&slide=id.p
  - 전체 Yolo 버전별 테스트 코드
    - https://github.com/roboflow/notebooks/tree/main/notebooks

- 대표적인 객체 탐지 모델
- 하나의 컨볼루션 네트워크를 통해 대상의 위치(바운딩박스)와 클래스(분류)를 한번에 예측 (1 state dectection 알고리즘)
  
  - 이미지를 S x S 그리드로 분할 (초기 논문 S = 7)
  - 이미지 전체를 신경망에 넣고 특징 추출을 통해 예측 텐서(그리드별 바운딩박스 정보, 신뢰점수, 분류 클래스 확률)를 생성
  - 그리드별 예측 정보를 기반으로 바운딩박스 조정 및 분류 작업 수행
  - 각 그리드 셀은 B 개의 바운딩박스와 바운딩박스에대한 신뢰점수를 가짐
  - 각 그리드 셀은 C 개의 분류 클래스 확률을 가짐
  - 각 바운딩박스는 왼쪽상단 좌표 (x, y)와 폭과 높이 (w, h), 실뢰점수를 가짐

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo01.png" width=50%> </center>

- 현재 YoloV8까지 공개

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo2.png" width=60%> </center>

### YoloV1
- Josept Redmon이 2015년에 yolov1 논문을 발표하고 공개
- convolutional layer로 이미지로부터 특징을 추출하고, FC layer로 바운딩박스와 class 확률을 예측
- 24개의 convolutional layer와 2개의 fully connected layer로 구성
- GoogLeNet에서 영향을 받아 1x1 차원 감소 layer (Inception) 뒤에 3x3 convolutional layer를 이용


- Image → CNN → FC → PT(Prediction Tensor) 순서로 동작


- 네트워크 구조
  - VGG16 모델을 기반으로 224x224 크기의 해상도로 학습을 하고, 448x448 크기의 이미지에 대해서 Object Detection을 수행하도록 설계

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo03.png" width=50%> </center>

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo03_02.png" width=50%> </center>

- 단점
  - 여러 물체들이 겹쳐있으면 제대로 된 예측이 어려움
  - 물체가 작을 수록 정확도가 감소
  - 바운딩박스 형태가 data를 통해 학습되므로 새로운 형태의 바운딩박스의 경우 정확히 예측하지 못함   

### YoloV2

- YOLOv1 모델의 문제점을 보완하여 정확도를 높인 모델
- 특징
  - CNN에 Batch Normalization 적용 - 2% 향상
  - 학습 전 이미지 분류 모델을 큰 해상도의 이미지에 대해 fine-tuning 단계를 거쳐 고해상도 이미지로 CNN 신경망을 학습 - 4% 향상
  - Anchor Box의 개념 도입
    - 마지막 단의 Fully Connected layer을 떼어내고 Convolutional Network 형태로 prediction을 계산
    - 바운딩 박스의 좌표를 예측하기 보다는 사전에 정의한 앵커 박스에서 offset을 예측
    - Anchor Box의 핵심 : 사전에 크기와 비율이 모두 결정되어 있는 박스를 전제로, 학습을 통해서 이 박스의 위치나 크기를 세부 조정하는 것

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo04.png" width=40%> </center>

  - features map 크기를 7x7 에서 13x13으로 변경
  
  - Multi-Scale Training : 작은 물체도 추출하기 위해 여러 스케일의 이미지를 학습 (일정반복마다 입력이미지의 해상도를 변경)
  
- 네트워크 구조
  - Darknet 19 신경망을 구축하여 이용
  - VGG-16 신경망에서 대부분의 가중치가 쓰인 FC layer를 제거하여 가중치 파라미터 수를 낮춰줬기 때문에 속도가 향상

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo05.png" width=60%> </center>

### YoloV3

- Josept Redmon이 2018년 4월 제안

- shortcut connection이 추가되어 53개의 layer를 가지는 Darknet-53을 backbone network로 사용
- Darkenet-53은 ResNet-101보다 1.5배 빠르며, ResNet-152와 비슷한 성능을 보이지만 2배 이상 빠름

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo06.png" width=70%> </center>

- 일반적인 SSD와 같은 구조는, feature extractwor의 앞쪽에서 나온 feature map은 표현력이 부족
- 이를 보완하기 위해 우측처럼 다시 deconvolution으로 feature map의 크기를 확장시켜 high-level feature를 뽑도록 함
- 왼쪽과 오른쪽의 feature map을 concat해서(왼쪽에서 위치정보 등을 갖고오는 식으로) 표현력을 향상시켜 사용함

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo07.png" width=70%> </center>

### YoloV4

- AlexeyBochkousky에 의해 2020년 4월 발표
- YoloV3에 비해 AP, FPS가 각각 10%, 12% 증가
- CSPNet 기반의 backbone(CSPDarkNet53)을 설계하여 사용
- 다양한 기법을 적용항 성능 향상 : WRC (Weighted-Residual-Connections), CSP (Cross-Stage-Partial-Connections), CmBN (Cross mini-Batch Normalizations), SAT (Self-Adversarial-Training), Mish Activation, Mosaic Data Agumentation, Drop Block Regularization, CIOU Loss

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo08.png" width=50%> </center>

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo09.png" width=50%> </center>

### YoloV5


- GlennJocher에 의해 2020년 6월 발표
- YoloV4에 비해 낮은 용량과 빠른 속도, 성능은 비슷 (FPS : 140, mAP : 89.5)
- YoloV4와 같은 CSPNet 기반의 backbone을 설계하여 사용
- Darknet이 아닌 PyTorch 구현이기 때문에, 이전 버전들과 다름

- 다른 yolo 모델들과 달리 크기별로 구분 : Yolov5s, Yolov5m, YOlov5l, Yolov5x 등/

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo10.png" width=40%> </center>

### YoloV7

- Chien-Yao Wang에 의해 2022년 7월 발표
- E-ELAN 네트워크를 사용

- YOLO 제품군의 새로운 물체 탐지기로 현재까지 가장 빠르고 정확한 실시간 물체 탐지기
  
- 달라진 점
  - 아키텍처 개선
  - 훈련가능한 BoF (Bag of Freebies)
  - 인간 포즈 추정 모델을 포함하는 첫번째 YOLO 모델  
  
  - bag-of-freebies 방법을 사용한 성능 향상
     - model reparameterization : 훈련시 여러 개의 레이어 (Conv, BN)들을 학습하고 추론 시에는 해당 레이어들을 하나의 레이어로 합침 (RepVGG를 수정한 형태)
     - Label assignment : 모델의 에측, ground truth (예측되기를 원하는 라벨)의 분포를 고려해서 새로운 soft label를 만듬 (ATSS, OTA, SimOTA 방법)

  - 파라미터와 계산을 고려한 model scaling 방법 사용

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo11.png" width=70%> </center>

### YoloV6

- 2022년 9월 발표된 버전
- 여러 방법을 이용하여 알고리즘의 효율을 높이고, 특히 시스템에 탑재하기 위한 Quantization과 distillation 방식도 일부 도입하여 성능 향상

- 백본은 EfficientRep Backbone이 사용
  - Neck 부분은 Rep-PAN이 사용
  - head는 Efficient decoupled head가 사용

- CSPstackRep Block = CSP(cross stage partial) + RepVGG 방식이 backbone에 사용

- 소형 모델은 일반 단일 경로 백본을 특징으로 하고 대형 모델은 효율적인 다중 분기 블록을 기반으로 구축

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo12.png" width=70%> </center>

### YoloV8

- 2023년 1월 발표된 버전
- YOLO 모델을 위한 완전히 새로운 리포지토리를 출시하여 개체 감지
- 인스턴스 세분화 및 이미지 분류 모델을 train하기 위한 통합 프레임워크로 구축됨

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_yolo13.png" width=70%> </center>

### Data annotation 기법 <font size=2>(참고 : https://blog.naver.com/datahive/222224206050)</font>

- 메타데이터를 데이터 셋에 추가하는 작업
  - 인공지능이  데이터의 내용을 이해할 수 있도록 주석을 달아주는 작업
  
- bounding box : 이미지 혹은 영상 안 객체의 가장자리에 딱 맞춘 사각형 틀을 그려주는 것
  - 자율주행 차량이 실제 시나리오에서 주변 환경과 모든 물체를 인식하고 이해하도록 훈련하는데 활용
  - 장점 : 쉽고 빠르게 데이터를 가공
  - 단점 : 객체의 유형과 위치에 따라 바운딩 박스 안에 해당 객체에 속하지 않는 픽셀이 포함될 수 있음
  - 난이도 : ★★

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/106_bounding_box.png" width=50%> </center>  


- polygon : 길 위에 있는 보행자나 자전거같이 규칙적이지 않은 형태의 객체를 다각형 형태로 영역을 지정
  - 훨씬 정확하게 객체에 속한 픽셀을 어노테이션 할 수 있는 기법
  - 작업 속도가 약간 느림
  - 장점 : 객체의 윤곽을 정밀하게 선택할 수 있어, 인공지능에게 그 물체의 크기와 형태를 정확하게 인식시킬 수 있음
  - 단점 : 겹쳐져 있는 객체에 폴리곤 기법을 적용하는 경우엔 목표물을 정확히 인식하기 어려움
  - 난이도 : ★★

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/106_polygon.png" width=50%> </center>    

- polyline : 객체 테두리에 여러 점을 찍어 선으로 구성되어 나타내는 기법
  - 동일한 장소에서 시작하고 종료할 필요가 없는 형태를 추적해야 할 때 유용
  - 자율 주행 자동차의 차선 탐지 훈련, 물류창고의 컨베이어 벨트 탐지 훈련 등에 활용
  - 장점 : 전선 감지, 차선 감지 등 직선 혹은 곡선을 추적해야 하는 사례에 매우 유용
  - 단점 : 이미지 상 객체의 선이 1픽셀 너비에 가까운 경우에만 작동하기 때문에 객체의 선 너비가 넓은 경우엔 폴리곤 기법으로 대체
  - 난이도 : ★★

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/106_polyline.PNG" width=50%> </center>      

- point : 이미지 속 객체의 개수를 계산하거나 군중 속에 있는 사람을 선택하는 등 이미지 상 단일 픽셀을 찾아내는 경우에 매우 유용
  - 장점 : 작업 방법이 매우 쉽고 간단하며 그림판 같은 매우 간단한 툴을 이용 가능
  - 단점 : 객체의 윤곽이 명확하지 않은 이미지에는 적용하기 어려움
  - 난이도 : ★★

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/106_point.PNG" width=50%> </center>    

- cuboid : 바운딩 박스와 굉장히 비슷한 방식의 기법으로 3D 형태
  - 길이, 너비뿐만 아니라 폭까지 표시할 수 있어 2D 이미지만으로는 부족한 정밀도를 높일 수 있음
  - 산업로봇과 같은 기계가 물체를 더욱 정확하게 인식할 수 있게 하는 경우에 흔히 사용
  - 장점 : 객체의 깊이에 대한 정보가 제공되기 때문에 3D 환경에서 중요한 정보들을 식별
  - 단점 : 객체가 불규칙한 형태를 가지거나 가려진 부분이 있는 경우, 잘린 부분이 있는 경우엔 작업이 어려움
  - 난이도 : ★★★★

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/106_cuboid.PNG" width=50%> </center>    

- semantic segmentation : 이미지에 있는 모든 픽셀을 채색하고 해당하는 클래스로 분류하는 방식
  - 자율 주행 자동차, 의료 영상 분석, 산업 검사, 위성 영상, 로봇 비전 등에 유용하게 활용
  - 이미지 속 모든 장면과 상황을 인식할 수 있게 가공하는 고차원의 방법
  - 장점 : 기계가 해당 장면을 왁벽히 이해시킬 수 있음
  - 단점 : 이미지의 모든 픽셀을 해당하는 클래스로 분류해야 하기 때문에 많은 작업이 필요
  - 난이도 : ★★★★

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/106_segmentation.PNG" width=30%> </center>    

# labelimg를 이용하여 customized 데이터셋 만들기

- 라벨링할 이미지를 원하는 폴더에 저장한다 (./images/yoloimg)

- 라벨링 툴을 이용하여 이미지에 포함된 개체를 라벨링
  - LabelImg, LabelMe, Label Studio, Diffgram, ImageTagger, VIA, Make Sense, COCO Annotator, DataTurks 등
  - roboflow : 라벨링툴 및 데이터셋 제공
    - https://public.roboflow.com/object-detection
  - CVAT : 크롬에서 실행, 강력한 최신 기능 지원
    - https://github.com/openvinotoolkit/cvat

# Roboflow을 이용한 데이터 라벨링

In [ ]:
%cd "drive/MyDrive/Colab Notebooks/23.12.18딥러닝"

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/23.12.18딥러닝'
/content/drive/MyDrive/Colab Notebooks/23.12.18딥러닝/yolo_project


In [ ]:
# yolo project 폴더 생성
!mkdir ./yolo_project/

In [ ]:
%cd ./yolo_project/

/content/drive/MyDrive/Colab Notebooks/23.12.18딥러닝/yolo_project


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Total 1197 (delta 0), reused 0 (delta 0), pack-reused 1197
Receiving objects: 100% (1197/1197), 74.23 MiB | 13.34 MiB/s, done.
Resolving deltas: 100% (520/520), done.
Updating files: 100% (108/108), done.


In [ ]:
# 데이터용 폴더 생성
!mkdir ./yolov7/data/catdog

In [ ]:
!curl -L "https://app.roboflow.com/ds/H0A00ZYoL0?key=Ldzfnv2xNM" > roboflow.zip; unzip roboflow.zip -d"./yolov7/data/catdog";rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   2527      0 --:--:-- --:--:-- --:--:--  2530
100  107M  100  107M    0     0  50.6M      0  0:00:02  0:00:02 --:--:-- 73.6M
Archive:  roboflow.zip
 extracting: ./yolov7/data/catdog/README.dataset.txt  
 extracting: ./yolov7/data/catdog/README.roboflow.txt  
 extracting: ./yolov7/data/catdog/data.yaml  
   creating: ./yolov7/data/catdog/test/
   creating: ./yolov7/data/catdog/test/images/
 extracting: ./yolov7/data/catdog/test/images/103_jpeg.rf.ee82e3895175f2da7d1272e767879cbc.jpg  
 extracting: ./yolov7/data/catdog/test/images/118_jpeg.rf.27cb341493e0bd6c074fcf64669fb83e.jpg  
 extracting: ./yolov7/data/catdog/test/images/134_jpeg.rf.13d58101f43f8cc511795a936f0de7c7.jpg  
 extracting: ./yolov7/data/catdog/test/images/137_jpeg.rf.6cad1b2a72d47011410f4bd188f42f04.jpg  
 extracting: ./yolov7/data/ca

In [ ]:
# yolov7관련 라이브러리 설치
!pip install -r ./yolov7/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00


In [ ]:
# 기본학습
!python ./yolov7/train.py --workers 8 --device 0 --batch-size 16 --data ./yolov7/data/catdog/data.yaml\
 --img 640 640 --cfg ./yolov7/cfg/training/yolov7.yaml --weights ''\
 --name yolov7 \
 --hyp ./yolov7/data/hyp.scratch.p5.yaml --epochs 5

2023-12-21 01:43:25.712927: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 01:43:25.712987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 01:43:25.714445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 01:43:25.722411: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 01:43:27.157293: W tensorflow/compiler/tf2

In [18]:
# 전이학습(transfer learning) : 사전에 학습된 모델을 가져와 내 데이터로 추가 학습하는 기법
# 장점 : 학습시간을 단축시키고 상대적으로 작은 데이터 크기로도 괜찮은 성능을 기대할 수 있다.
!python ./yolov7/train.py --workers 8 --device 0 --batch-size 16 --data ./yolov7/data/catdog/data.yaml\
 --img 640 640 --cfg ./yolov7/cfg/training/yolov7x.yaml --weights './yolov7x_training.pt'\
 --name yolov7 \
 --hyp ./yolov7/data/hyp.scratch.p5.yaml --epochs 10

2023-12-21 02:23:36.728213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 02:23:36.728302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 02:23:36.730516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 02:23:36.743591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 02:23:38.465609: W tensorflow/compiler/tf2

In [19]:
!python ./yolov7/train.py --workers 8 --device 0 --batch-size 16 --data ./yolov7/data/catdog/data.yaml\
 --img 640 640 --cfg ./yolov7/cfg/training/yolov7x.yaml --weights './runs/train/yolov74/weights/last.pt'\
 --name yolov7 \
 --hyp ./yolov7/data/hyp.scratch.p5.yaml --epochs 10

2023-12-21 03:03:27.098663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 03:03:27.098720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 03:03:27.100191: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 03:03:27.108412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 03:03:29.294960: W tensorflow/compiler/tf2

In [22]:
# 사진 예측하기
!python ./yolov7/detect.py --weights ./runs/train/yolov74/weights/best.pt \
--conf 0.25 --img-size 640 \
--source ./test

Namespace(weights=['./runs/train/yolov74/weights/best.pt'], source='./test', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 2023-12-21 torch 2.1.0+cu121 CUDA:0 (Tesla T4, 15102.0625MB)

Fusing layers... 
IDetect.fuse
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 362 layers, 70789182 parameters, 0 gradients, 188.0 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

Done. (35.0ms) Inference, (92.4ms) NMS
 The image with the result is saved in: runs/detect/exp3/mong

#### 성능평가

- IoU (Intersection over union) : 예측된 바운딩 박스와 사용자가 설정한 바운딩 박스간 중첩되는 부분의 면적을 측정해서 중첩된 면접을 합집합의 면적으로 나눈 것

<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_iou.jpg" width=40%></center>

- AP (Average Precision) : 컴퓨터 비전 분야에서 물체 검출 및 이미지 분류 알고리즘의 성능 평가 기준
  - AP는 precision-recall 그래프에서 그래프 선 아래쪽의 면적으로 계산
  - 보통 계산 전에 PR 곡선을 계단식으로 감소하는 그래프가 되게 하기 위해서 변환하고 넓이를 구함 (적색)
  - AP = 왼쪽 큰 사각형의 넓이 + 오른쪽 작은 사각형의 넓이 = 1*0.33 + 0.88*(0.47-0.33) = 0.4532
  
<center><img src="https://arome1004.cafe24.com/images/cv_project/lecture_image/105_ap.png" width=40%></center>

- mAP (Mean Average Precision) : 물체가 여러 개인 경우 각 클래스별로 AP를 구하고 합을 구한 후에 물체의 개수로 나눈 값

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>정리하기</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ 객체 탐지(object detection) : 컴퓨터 비전과 이미지 처리와 관련된 컴퓨터 기술로서, 디지털 이미지와 비디오에서 관심 객체(예: 인간, 건물, 자동차 등)의 위치를 감지하는 작업

○ 객체 분류 : 탐지된 객체가 무엇인지 판단하고 분류하는 작업

○ 객체 탐지 및 분류 알고리즘

  - 머신러닝 기반 접근 : SIFT, HOG 등
  
  - 딥러닝 기반 접근 : R-CNN, Fast R-CNN, Faster R-CNN, cascade R-CNN, SSD, YOLO, RefineDet 등

        
○ Two-Stage Detector : Classification, Regional Proposal을 순차적으로 수행하여 결과를 얻는 방법 (정확도는 우수하지만 속도가 느림) - R-CNN, Fast R-CNN, Faster R-CNN 등
        
○ One-Stage Detector : Classification, Regional Proposal을 동시에 수행하여 결과를 얻는 방법 (속도 개선) - YOLO, SSD, RetinaNet 등

        
○ 바운딩 박스 (Bounding Box) : 객체 탐지 모델에서 바운딩 박스는 타겟 위치를 특정하기 위해 사용

○ Yolo : YoloV1, YoloV2, YoloV3, YoloV4, YoloV5, YoloV7 등           
        
</font></td></tr>   
</table>